# DATA_EXPLORATION

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
#importing local modules
sys.path.insert(0,'../Scripts/')
from cleaner import DataCleaner
from data_information import DataInfo
from Data_logger import CreateLogger
from data_manipulation import DataManipulator
from Graphs import draw
from Results import result_picker



### Loading Data

In [16]:
#loading train data
res=result_picker()

train_df = pd.read_csv('../data/train.csv')
train_data_info = DataInfo(train_df, deep=True)
train_data_info.get_information()

C:\Users\user\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


DataFrame Information: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   Store          1017209 non-null  int64 
 1   DayOfWeek      1017209 non-null  int64 
 2   Date           1017209 non-null  object
 3   Sales          1017209 non-null  int64 
 4   Customers      1017209 non-null  int64 
 5   Open           1017209 non-null  int64 
 6   Promo          1017209 non-null  int64 
 7   StateHoliday   1017209 non-null  object
 8   SchoolHoliday  1017209 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 69.8+ MB


In [4]:
#Loading the test data
test_df = pd.read_csv('../data/test.csv')
test_data_info = DataInfo(test_df, deep=True)
test_data_info.get_information()

DataFrame Information: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41088 entries, 0 to 41087
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             41088 non-null  int64  
 1   Store          41088 non-null  int64  
 2   DayOfWeek      41088 non-null  int64  
 3   Date           41088 non-null  object 
 4   Open           41077 non-null  float64
 5   Promo          41088 non-null  int64  
 6   StateHoliday   41088 non-null  object 
 7   SchoolHoliday  41088 non-null  int64  
dtypes: float64(1), int64(5), object(2)
memory usage: 2.5+ MB


In [10]:
#missing values for the train data
missing_train = train_data_info.get_column_based_missing_percentage()
res.add_data('missing_data',missing_train)

#missing values for the test data
missing_test = test_data_info.get_column_based_missing_percentage()
res.add_data('missing_data',missing_test)

missing_train




,total_missing_values,missing_percentage
Store,0,0.0 %
DayOfWeek,0,0.0 %
Date,0,0.0 %
Sales,0,0.0 %
Customers,0,0.0 %
Open,0,0.0 %
Promo,0,0.0 %
StateHoliday,0,0.0 %
SchoolHoliday,0,0.0 %


In [11]:
missing_test

,total_missing_values,missing_percentage
Id,0,0.0 %
Store,0,0.0 %
DayOfWeek,0,0.0 %
Date,0,0.0 %
Open,11,0.03 %
Promo,0,0.0 %
StateHoliday,0,0.0 %
SchoolHoliday,0,0.0 %


#### Train and Test data has a compromizable (almost no) missing data

In [13]:
#Loading the store data
store_df = pd.read_csv('../data/store.csv')
store_data_info = DataInfo(store_df, deep=True)
store_data_info.get_information()

DataFrame Information: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Store                      1115 non-null   int64  
 1   StoreType                  1115 non-null   object 
 2   Assortment                 1115 non-null   object 
 3   CompetitionDistance        1112 non-null   float64
 4   CompetitionOpenSinceMonth  761 non-null    float64
 5   CompetitionOpenSinceYear   761 non-null    float64
 6   Promo2                     1115 non-null   int64  
 7   Promo2SinceWeek            571 non-null    float64
 8   Promo2SinceYear            571 non-null    float64
 9   PromoInterval              571 non-null    object 
dtypes: float64(5), int64(2), object(3)
memory usage: 87.2+ KB


In [14]:
missing_store = store_data_info.get_column_based_missing_percentage()
res.add_data('missing_data',missing_store)

missing_store

,total_missing_values,missing_percentage
Store,0,0.0 %
StoreType,0,0.0 %
Assortment,0,0.0 %
CompetitionDistance,3,0.27 %
CompetitionOpenSinceMonth,354,31.75 %
CompetitionOpenSinceYear,354,31.75 %
Promo2,0,0.0 %
Promo2SinceWeek,544,48.79 %
Promo2SinceYear,544,48.79 %
PromoInterval,544,48.79 %


#### Promos2sinceweek, Promo2SinceYear, PromoInterval have almost missing data of 50%, which are irrelevant for decision making so will be omitted

In [25]:
#train_df['StateHoliday'] = train_df['StateHoliday'].apply(lambda x: 0 if x == '0' else x)
#test_df['StateHoliday'] = test_df['StateHoliday'].apply(lambda x: 0 if x == '0' else x)

In [40]:
#given the object type date so we have to change it to datatime for seasonality analysis
train_cleaner = DataCleaner(train_df)
test_cleaner = DataCleaner(test_df)

In [52]:
train_cleaner.change_column_to_date_type('Date')
test_cleaner.change_column_to_date_type('Date')

train_cleaner.df.dtypes

Store                     int64
DayOfWeek                 int64
Date             datetime64[ns]
Sales                     int64
Customers                 int64
Open                      int64
Promo                     int64
StateHoliday             object
SchoolHoliday             int64
dtype: object

In [53]:
test_cleaner.df.dtypes

Id                        int64
Store                     int64
DayOfWeek                 int64
Date             datetime64[ns]
Open                    float64
Promo                     int64
StateHoliday             object
SchoolHoliday             int64
dtype: object

In [59]:
train_cleaner.df.head()

,Store,DayOfWeek,Year,Month,Day,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015,7,31,5263,555,1,1,0,1
1,2,5,2015,7,31,6064,625,1,1,0,1
2,3,5,2015,7,31,8314,821,1,1,0,1
3,4,5,2015,7,31,13995,1498,1,1,0,1
4,5,5,2015,7,31,4822,559,1,1,0,1
